## <center>Курсовой проект<a class="anchor" id="course_project"></a><center>

Ссылка на соревнование - https://www.kaggle.com/c/geekbrains-competitive-data-analysis/data

Постановка задачи.

Для построения модели в данном соревновании, сначала нужно будет собрать выборку для обучения модели. Формат соревнования очень похож на то, как в промышленности Data Scinetist'ы строят алгоритмы: сначала нужно провести анализ данных, собрать выборку и после этого строить модели. В соревновании представлены 4 типы источника данных, которые могут быть интерпретированы как таблицы в базе данных. Некоторые источники данных уже готовы для моделирования, представлены в агрерированном виде. Другие источники данных требуется представить в удобном для модели виде.

Описание источников данных:

    train.csv - пары "заявка - целевая переменная", для этой выборки нужно собрать признаки и обучить модель;
    test.csv - пары "заявки - прогнозное значение", для этой выборки нужно собрать признаки и построить прогнозы;
    bki.csv - данные БКИ о предыдущих кредитах клиента;
    client_profile.csv - клиентский профиль, некоторые знания, которые есть у компании о клиенте;
    payments.csv - история платежей клиента;
    applications_history.csv - история предыдущих заявок клиента.


In [1]:
import re
import numpy as np
import pandas as pd
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt
import missingno as msno
from sklearn import metrics
from datetime import timedelta
from datetime import datetime
import xgboost as xgb, lightgbm as lgbm, catboost as catb

from sklearn.model_selection import train_test_split
# from sunbird.categorical_encoding import frequency_encoding
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
import scipy.stats as st

import warnings
warnings.simplefilter("ignore")

import warnings
from tqdm import tqdm
from typing import List, Tuple

from scipy.stats import ttest_rel

# from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score

warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
train_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/train.csv')
train_df.head(4)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash
2,123526683,0,Cash
3,123710391,1,Cash


In [3]:
app_history_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/applications_history.csv')
app_history_df.head(4)

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,...,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,...,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,...,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0
2,49791680,123445379,Cash,15060.735,112500.0,136444.5,NaN,112500.0,Approved,301,...,x-sell,-1,12.0,high,365243.0,271.0,59.0,365243.0,365243.0,1.0
3,50087457,123499497,Cash,47041.335,450000.0,470790.0,NaN,450000.0,Approved,512,...,x-sell,-1,12.0,middle,365243.0,482.0,152.0,182.0,177.0,1.0


In [4]:
bki_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/bki.csv')
bki_df.head(4)

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,123538884,5223613,Active,currency 1,718.0,0,377.0,NaN,19386.81,0,675000.00,320265.495,0.0,0.0,Consumer credit,39.0,NaN
1,123436670,6207544,Closed,currency 1,696.0,0,511.0,511.0,0.00,0,93111.66,0.000,0.0,0.0,Consumer credit,505.0,NaN
2,123589020,6326395,Closed,currency 1,165.0,0,149.0,160.0,NaN,0,36000.00,0.000,0.0,0.0,Consumer credit,150.0,0.0
3,123494590,6606618,Active,currency 1,55.0,0,310.0,NaN,NaN,0,38664.00,37858.500,NaN,0.0,Consumer credit,15.0,NaN


In [5]:
client_profile_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/client_profile.csv')
client_profile_df.head(4)

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,...,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,...,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0
2,123501780,M,1,427500.0,239850.0,23850.0,Incomplete higher,Married,0.072508,14387,...,3.0,0.409017,0.738159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123588799,M,0,112500.0,254700.0,17149.5,Secondary / secondary special,Married,0.019101,14273,...,2.0,NaN,0.308994,0.590233,0.0,0.0,0.0,0.0,0.0,3.0


In [6]:
payments_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/payments.csv')
payments_df.head(4)

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,49011181,123664960,1.0,5,1002.0,1015.0,12156.615,12156.615
1,48683432,123497205,1.0,13,442.0,432.0,18392.535,10047.645
2,48652024,123749925,1.0,10,8.0,23.0,5499.945,5499.945
3,48398897,123550846,0.0,82,398.0,398.0,7082.145,7082.145


In [7]:
payments_df['APPLICATION_NUMBER'].value_counts()

123619703    37
123486520    33
123449869    31
123682859    31
123632163    31
             ..
123731958     1
123778085     1
123722215     1
123639711     1
123560468     1
Name: APPLICATION_NUMBER, Length: 264726, dtype: int64

Проведем анализ признаков. 

Постараемся собрать из несколькоих таблиц одну.

посмотрим что есть по по клиентам. 

посмотреть сколько кредитов в среднем у человека есть. 

сколько кредитов погашено, а сколько осталось.




In [8]:
app_history_df.loc[((app_history_df['NAME_CONTRACT_STATUS']=='Approved') & app_history_df['NFLAG_INSURED_ON_APPROVAL'].isna())]

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,...,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
18,49661323,123719644,Cash,10181.700,180000.0,180000.0,NaN,180000.0,Approved,700,...,x-sell,6,24.0,low_normal,NaN,NaN,NaN,NaN,NaN,NaN
47,48743952,123559076,Cash,75897.630,2025000.0,2241513.0,NaN,2025000.0,Approved,524,...,x-sell,-1,54.0,low_normal,NaN,NaN,NaN,NaN,NaN,NaN
74,48653311,123692762,NaN,2652.705,19611.0,14170.5,6111.0,19611.0,Approved,1525,...,XNA,2078,6.0,middle,NaN,NaN,NaN,NaN,NaN,NaN
172,48650361,123659969,Cash,10061.910,112500.0,127350.0,NaN,112500.0,Approved,631,...,walk-in,4,24.0,high,NaN,NaN,NaN,NaN,NaN,NaN
239,49488458,123626811,NaN,41383.710,234243.0,234243.0,0.0,234243.0,Approved,269,...,XNA,100,6.0,low_normal,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670146,48882557,123639067,Cash,83758.275,2025000.0,2169342.0,NaN,2025000.0,Approved,678,...,x-sell,-1,36.0,low_normal,NaN,NaN,NaN,NaN,NaN,NaN
1670150,48825150,123493032,Cash,18852.435,90000.0,92970.0,NaN,90000.0,Approved,253,...,x-sell,-1,6.0,high,NaN,NaN,NaN,NaN,NaN,NaN
1670159,49622726,123716694,Cash,20380.500,450000.0,450000.0,NaN,450000.0,Approved,706,...,x-sell,-1,48.0,middle,NaN,NaN,NaN,NaN,NaN,NaN
1670175,48444623,123571293,Cash,9740.250,135000.0,135000.0,NaN,135000.0,Approved,1010,...,x-sell,-1,18.0,middle,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
app_history_df['NFLAG_INSURED_ON_APPROVAL'].isna()

0          False
1          False
2          False
3          False
4           True
           ...  
1670209    False
1670210    False
1670211    False
1670212    False
1670213    False
Name: NFLAG_INSURED_ON_APPROVAL, Length: 1670214, dtype: bool

In [10]:
# Проведем анализ таблицы БКИ 

# Попробуем выявить зависимости.  

# Узнаем что за собой скрывают пропуски.

In [11]:
bki_df['CREDIT_CURRENCY'].value_counts()

currency 1    944492
currency 2       647
currency 3        91
currency 4         4
Name: CREDIT_CURRENCY, dtype: int64

In [12]:
bki_df['CREDIT_ACTIVE'].value_counts()

Closed      594315
Active      347323
Sold          3583
Bad debt        13
Name: CREDIT_ACTIVE, dtype: int64

In [13]:
# Объединим таблицы. 
train_df.merge(client_profile_df,how='inner', on='APPLICATION_NUMBER')

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,0,Cash,M,1,157500.0,855000.0,25128.0,Secondary / secondary special,Married,...,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0
1,123526683,0,Cash,F,0,135000.0,1006920.0,42660.0,Higher education,Married,...,2.0,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0
2,123710391,1,Cash,M,0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,...,2.0,NaN,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0
3,123718821,0,Cash,F,1,180000.0,755190.0,36459.0,Secondary / secondary special,Married,...,3.0,NaN,0.548504,0.331251,1.0,0.0,0.0,0.0,0.0,2.0
4,123544624,0,Cash,M,0,202500.0,1078200.0,38331.0,Secondary / secondary special,Married,...,2.0,NaN,0.547421,0.740799,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89534,123595479,0,Cash,F,0,225000.0,990432.0,32863.5,Secondary / secondary special,Separated,...,1.0,0.768198,0.713399,0.529890,0.0,0.0,0.0,0.0,0.0,0.0
89535,123426326,0,Cash,F,2,135000.0,995643.0,32904.0,Secondary / secondary special,Married,...,4.0,0.410920,0.592042,0.570917,0.0,0.0,0.0,0.0,0.0,1.0
89536,123672463,0,Cash,F,0,175500.0,269550.0,12618.0,Secondary / secondary special,Married,...,2.0,0.528096,0.471774,0.581484,0.0,0.0,0.0,0.0,0.0,2.0
89537,123554358,0,Cash,F,2,270000.0,1024740.0,49428.0,Incomplete higher,Married,...,4.0,0.653826,0.651227,0.558507,0.0,0.0,0.0,1.0,0.0,1.0


In [14]:
# client_profile_df.loc[(client_profile_df['APPLICATION_NUMBER']==123590329)]

In [15]:
#  bki_df.loc[(bki_df['APPLICATION_NUMBER']==123723001)]

In [16]:
# app_history_df.loc[(app_history_df['APPLICATION_NUMBER']==123723001)]

In [17]:
payments_df.loc[(payments_df['APPLICATION_NUMBER']==123723001)]

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
752132,49650758,123723001,1.0,4,70.0,83.0,4998.645,4998.645


In [18]:
bki_df['APPLICATION_NUMBER'].value_counts()

123444199    63
123493043    51
123641404    50
123603494    39
123574982    36
             ..
123503595     1
123530515     1
123664592     1
123775289     1
123685733     1
Name: APPLICATION_NUMBER, Length: 273131, dtype: int64

In [19]:
payments_df['APPLICATION_NUMBER'].value_counts()

123619703    37
123486520    33
123449869    31
123682859    31
123632163    31
             ..
123731958     1
123778085     1
123722215     1
123639711     1
123560468     1
Name: APPLICATION_NUMBER, Length: 264726, dtype: int64

In [20]:
bki_df['CREDIT_ACTIVE'].value_counts()

Closed      594315
Active      347323
Sold          3583
Bad debt        13
Name: CREDIT_ACTIVE, dtype: int64

In [21]:
credit_curency = {'currency 1':0, 'currency 2':1, 'currency 3':2, 'currency 4':3}

In [22]:
bki_df['CREDIT_CURRENCY'] = bki_df['CREDIT_CURRENCY'].apply(lambda x: credit_curency[x])

In [23]:
credit_type = bki_df['CREDIT_TYPE'].value_counts()

In [24]:
# def creat_features(app_number):
#     bki = bki_df.loc[(bki_df['APPLICATION_NUMBER']==app_number)]

#     credit_problems = np.sum(bki['CNT_CREDIT_PROLONG'])

#     active_credits = 0
#     closed_credits = 0
#     sold_credits = 0
#     bad_debt = 0

#     for status in bki['CREDIT_ACTIVE']:
#         if status == 'Active':
#             active_credits +=1
#         if status == 'Closed':
#             closed_credits +=1
#         if status == 'Sold':
#             sold_credits +=1
#         if status == 'Bad debt':
#             bad_debt +=1  

#     active_credits, closed_credits, sold_credits, bad_debt  
#     credits_count = len(bki)
#     curency_mean = np.mean(bki['CREDIT_CURRENCY'])
    
#     bki_dic['APPLICATION_NUMBER'].append(app_number)
#     bki_dic['active_credits'].append(active_credits)
#     bki_dic['closed_credits'].append(closed_credits)
#     bki_dic['sold_credits'].append(sold_credits)
#     bki_dic['bad_debt'].append(bad_debt)
#     bki_dic['credit_problems'].append(credit_problems)
#     bki_dic['credits_count'].append(credits_count)
#     bki_dic['curency_mean'].append(curency_mean)
#     bki_dic['curency_mean'].append(curency_mean)

# # bki_unic = bki_df['APPLICATION_NUMBER'].unique()

# # for app_number in tqdm(bki_unic):
# #     creat_features(app_number)

# # bki_statistic = pd.DataFrame(bki_dic)
# # bki_statistic.to_csv('bki_statistic.csv',index=False)
# bki_statistic = pd.read_csv('bki_statistic.csv')
# np.max(bki_statistic['APPLICATION_NUMBER'].value_counts().values)
# bki_statistic.head(5)

In [25]:
# bki_statistic['credit_problems'].value_counts()

In [26]:
train_df['TARGET'].value_counts()

0    101196
1      8897
Name: TARGET, dtype: int64

In [27]:
# Сгруппируем признаки из набора данных так, чтобы получилось что-то адекватное.

In [28]:
categorical = []
numerical = []
for col, value in bki_df.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
    else:
        numerical.append(col)

In [29]:
categorical

['CREDIT_ACTIVE', 'CREDIT_TYPE']

In [30]:
client_profile_df['FAMILY_STATUS'].value_counts()

Married                 159846
Single / not married     36883
Civil marriage           24252
Separated                15984
Widow                    13033
Unknown                      2
Name: FAMILY_STATUS, dtype: int64

In [31]:
client_profile_df['EDUCATION_LEVEL'].value_counts()

Secondary / secondary special    177562
Higher education                  60864
Incomplete higher                  8357
Lower secondary                    3081
Academic degree                     136
Name: EDUCATION_LEVEL, dtype: int64

In [32]:
client_profile_df.loc[(client_profile_df['GENDER']=='XNA')]

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
23011,123506723,XNA,1,135000.0,405000.0,20250.0,Higher education,Married,0.035792,10647,...,3.0,0.404970,0.659185,0.076984,0.0,0.0,0.0,0.0,0.0,3.0
34379,123458998,XNA,0,207000.0,382500.0,19125.0,Secondary / secondary special,Married,0.020713,20232,...,2.0,NaN,0.295998,0.461482,0.0,0.0,0.0,0.0,0.0,1.0
81943,123461907,XNA,2,157500.0,270000.0,13500.0,Secondary / secondary special,Married,0.026392,13717,...,4.0,NaN,0.709205,0.310818,0.0,0.0,0.0,3.0,0.0,4.0
178975,123612981,XNA,0,247500.0,540000.0,27000.0,Incomplete higher,Civil marriage,0.035792,9649,...,2.0,0.653035,0.658620,0.360613,0.0,0.0,0.0,0.0,1.0,6.0


In [33]:
def get_freq_encoder(data,feature_names):
    for feature_name in feature_names:
        freq_encoder = data[feature_name].value_counts(normalize=True)
        data[f"{feature_name}_freq_enc"] = data[feature_name].map(freq_encoder)
    data.drop(feature_names, axis=1, inplace=True)
    return data

In [34]:
bki_dic = {
    'APPLICATION_NUMBER':[],
     'active_credits':[],
     'closed_credits':[],
     'sold_credits':[],
     'bad_debt':[],
     'credit_problems':[], 
     'credits_count':[],
     'curency_mean':[],
     'credit_type_mean':[]
}
 

In [35]:
bki_df = get_freq_encoder(bki_df, ['CREDIT_TYPE'])

In [36]:
def creat_features(app_number):
    bki = bki_df.loc[(bki_df['APPLICATION_NUMBER']==app_number)]

    credit_problems = np.sum(bki['CNT_CREDIT_PROLONG'])

    active_credits = 0
    closed_credits = 0
    sold_credits = 0
    bad_debt = 0

    for status in bki['CREDIT_ACTIVE']:
        if status == 'Active':
            active_credits +=1
        if status == 'Closed':
            closed_credits +=1
        if status == 'Sold':
            sold_credits +=1
        if status == 'Bad debt':
            bad_debt +=1  

    active_credits, closed_credits, sold_credits, bad_debt  
    credits_count = len(bki)
    curency_mean = np.mean(bki['CREDIT_CURRENCY'])
    credit_type_mean = np.mean(bki['CREDIT_TYPE_freq_enc'])
    bki_dic['APPLICATION_NUMBER'].append(app_number)
    bki_dic['active_credits'].append(active_credits)
    bki_dic['closed_credits'].append(closed_credits)
    bki_dic['sold_credits'].append(sold_credits)
    bki_dic['bad_debt'].append(bad_debt)
    bki_dic['credit_problems'].append(credit_problems)
    bki_dic['credits_count'].append(credits_count)
    bki_dic['curency_mean'].append(curency_mean)
    bki_dic['credit_type_mean'].append(credit_type_mean)


bki_unic = bki_df['APPLICATION_NUMBER'].unique()

for app_number in tqdm(bki_unic):
    creat_features(app_number)

bki_statistic = pd.DataFrame(bki_dic)
bki_statistic.to_csv('bki_statistic.csv',index=False)

# bki_statistic = pd.read_csv('bki_statistic.csv')
# np.max(bki_statistic['APPLICATION_NUMBER'].value_counts().values)
bki_statistic.head(5)

100%|██████████| 273131/273131 [06:02<00:00, 753.39it/s]


,APPLICATION_NUMBER,active_credits,closed_credits,sold_credits,bad_debt,credit_problems,credits_count,curency_mean,credit_type_mean
0,123538884,2,1,0,0,0,3,0.0,0.728318
1,123436670,5,3,0,0,0,8,0.0,0.420066
2,123589020,0,7,0,0,0,7,0.0,0.728318
3,123494590,2,1,0,0,0,3,0.0,0.563917
4,123446603,1,0,0,0,0,1,0.0,0.235114


In [37]:
# client_profile_df_1 = get_freq_encoder(client_profile_df, ['EDUCATION_LEVEL', 'FAMILY_STATUS'])

In [38]:
# client_profile_df_1

In [39]:
def category_to_digit(df, features):
    df = df.copy(deep=True)
    for i, feature in enumerate(features):
        # feature = str.replace(feature,' ','_')
        values_list = df[feature].value_counts()
        names = sorted(values_list.index)
        # names = sorted(names)
        for name in names:
            name = str.replace(name,' ','_')
            df.insert(3, f'{feature}_{name}', np.where((df[feature]==name),1,0), True)
    df.drop(features, axis=1, inplace=True)
    return df

In [40]:
def df_prepare(source_df, join_type ='left'):
    source_df = source_df.copy(deep=True)
    client_profile_df_1 = category_to_digit(client_profile_df, ['EDUCATION_LEVEL', 'FAMILY_STATUS'])
    # client_profile_df_1 = get_freq_encoder(client_profile_df, ['EDUCATION_LEVEL', 'FAMILY_STATUS'])
     # самые важные признаки (веса - по важности)
    client_profile_df_1["EXTERNAL_SCORING_RATING"] = (client_profile_df_1["EXTERNAL_SCORING_RATING_1"] * client_profile_df_1["EXTERNAL_SCORING_RATING_2"] * client_profile_df_1["EXTERNAL_SCORING_RATING_3"])
    client_profile_df_1["EXTERNAL_SCORING_RATING_SUM"] = (client_profile_df_1["EXTERNAL_SCORING_RATING_1"] + client_profile_df_1["EXTERNAL_SCORING_RATING_2"] + client_profile_df_1["EXTERNAL_SCORING_RATING_3"])
    client_profile_df_1["SOLVENCY"] = client_profile_df_1["TOTAL_SALARY"]/client_profile_df_1["AMOUNT_CREDIT"]
    client_profile_df_1["CREDIT_LONG"] = client_profile_df_1["AMOUNT_CREDIT"]/client_profile_df_1["AMOUNT_ANNUITY"]
   
    df = source_df.merge(client_profile_df_1, how=join_type, on='APPLICATION_NUMBER')
    df = df.merge(bki_statistic, how=join_type ,on='APPLICATION_NUMBER')
    
    df['GENDER'] = np.where(df['GENDER']=='M', 1, 0)
    df['AGE'] = df['AGE'].apply(lambda x: x / 365)
    df['DAYS_ON_LAST_JOB'] = df['DAYS_ON_LAST_JOB'].apply(lambda x: x / 365)

    # df = category_to_digit(df, ['NAME_CONTRACT_TYPE'])
    df = get_freq_encoder(df, ['NAME_CONTRACT_TYPE'])
    
    return df

In [41]:
categorical = []
numerical = []
for col, value in client_profile_df.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
    else:
        numerical.append(col)
categorical

['GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS']

In [42]:
# Попробуем обучить модель на этих данных.

In [43]:
def balance_df_by_target(df, target_name, method='over'):

    assert method in ['over', 'under', 'tomek', 'smote'], 'Неверный метод сэмплирования'
    
    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1
    if method == 'over':
        for i in range(disbalance_coeff):
            sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
            df = df.append(sample, ignore_index=True)
            
    elif method == 'under':
        df_ = df.copy()
        df = df_[df_[target_name] == minor_class_name]
        tmp = df_[df_[target_name] == major_class_name]
        df = df.append(tmp.iloc[
            np.random.randint(0, tmp.shape[0], target_counts[minor_class_name])
        ], ignore_index=True)

    elif method == 'tomek':
        from imblearn.under_sampling import TomekLinks
        tl = TomekLinks()
        X_tomek, y_tomek = tl.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_tomek, y_tomek], axis=1)
    
    elif method == 'smote':
        from imblearn.over_sampling import SMOTE
        smote = SMOTE()
        X_smote, y_smote = smote.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_smote, y_smote], axis=1)

    return df.sample(frac=1) 

In [44]:
def evalute_model(df, estimator: object, metric: callable, is_balanced=True):

 # Разделим выборки на тренировчную, тестовую и валидационную.
    x_train, x_test = train_test_split(df.drop(["APPLICATION_NUMBER", "TARGET"], axis=1), train_size=0.60, shuffle=False, random_state=1)
    y_train, y_test = train_test_split(df["TARGET"], train_size=0.60, shuffle=False, random_state=1,) # В данных имеется временная зависимость, поэтому пока перемешивать ничего не будем.

    x_test, x_val = train_test_split(x_test, train_size=0.50, shuffle=False, random_state=1,)
    y_test, y_val = train_test_split(y_test, train_size=0.50, shuffle=False, random_state=1,)

    if is_balanced:
      df_for_balancing = pd.concat([x_train, y_train], axis=1)
      df_balanced = balance_df_by_target(df_for_balancing, "TARGET", method='over')
          
      x_train = df_balanced.drop(columns="TARGET")
      y_train = df_balanced["TARGET"]

    estimator = estimator
    
    estimator.fit(
        X=x_train,
        y=y_train,
        eval_set=[(x_train, y_train), (x_val, y_val)],
        verbose=50
    )

    train_score =  metric(y_train, estimator.predict_proba(x_train)[:,1])
    val_score = metric(y_val, estimator.predict_proba(x_val)[:,1])
    test_score = metric(y_test, estimator.predict_proba(x_test)[:,1])
   
    print(f"\r\n Train-score: {round(train_score, 3)}, Val-score: {round(val_score, 3)}, Test-score: {round(test_score, 3)}")

    return estimator

In [45]:
lgbm_model = lgb.LGBMClassifier(
        boosting_type="gbdt",
        objective ="binary",
        metric = "auc",
        learning_rate = 0.1,
        n_estimators = 10000,
        reg_lambda = 50,
        max_depth = 4,
        random_state = 1,
        early_stopping_round=500
       )

In [46]:
catb_model = catb.CatBoostClassifier(silent=True, random_state=21,
                                    score_function='L2',
                                    depth = 7,
                                    eval_metric='AUC',
                                    use_best_model=True,
                                    boosting_type = 'Ordered', #  / Plain 
                                    l2_leaf_reg = 3, # -- регуляризация. 3.0 = по умолчанию. 
                                    # eta = 0.5,
                                    learning_rate=0.01,
                                    custom_metric=['Precision', 'Recall'],
                                    iterations=10000,
                                    early_stopping_rounds=500,
                                    # verbose=True
                                )

In [47]:
df = df_prepare(train_df,join_type='inner')

In [48]:
df.head(10)

,APPLICATION_NUMBER,TARGET,GENDER,CHILDRENS,FAMILY_STATUS_Widow,FAMILY_STATUS_Unknown,FAMILY_STATUS_Single_/_not_married,FAMILY_STATUS_Separated,FAMILY_STATUS_Married,FAMILY_STATUS_Civil_marriage,...,CREDIT_LONG,active_credits,closed_credits,sold_credits,bad_debt,credit_problems,credits_count,curency_mean,credit_type_mean,NAME_CONTRACT_TYPE_freq_enc
0,123687442,0,1,1,0,0,0,0,1,0,...,34.025788,0,1,0,0,0,1,0.0,0.728318,0.903846
1,123526683,0,0,0,0,0,0,0,1,0,...,23.603376,2,7,0,0,0,9,0.0,0.593788,0.903846
2,123710391,1,1,0,0,0,0,0,1,0,...,22.573164,0,1,0,0,0,1,0.0,0.728318,0.903846
3,123718821,0,0,1,0,0,0,0,1,0,...,20.713404,1,0,0,0,0,1,0.0,0.235114,0.903846
4,123544624,0,1,0,0,0,0,0,1,0,...,28.128669,1,0,0,0,0,1,0.0,0.235114,0.903846
5,123631154,0,1,0,0,0,0,0,0,0,...,8.841733,1,2,0,0,0,3,0.0,0.728318,0.903846
6,123702544,0,1,0,0,0,0,0,0,0,...,20.245072,2,2,0,0,0,4,0.0,0.605017,0.903846
7,123660201,0,1,0,0,0,0,0,0,0,...,9.261898,1,1,0,0,0,2,0.0,0.728318,0.903846
8,123676382,0,1,0,0,0,0,0,1,0,...,26.197664,2,0,0,0,0,2,0.0,0.481716,0.903846
9,123728314,0,0,0,0,0,0,1,0,0,...,30.999742,0,1,0,0,0,1,0.0,0.728318,0.903846


In [49]:
# Оптимизируем данные. обернем все в pipline. 

# Подготовка данных для датасета.

In [50]:
df = df_prepare(train_df,join_type='left')

# Разделим выборки на тренировчную, тестовую и валидационную.
x_train, x_test = train_test_split(df.drop(["APPLICATION_NUMBER", "TARGET"], axis=1), train_size=0.80, shuffle=False, random_state=1)
y_train, y_test = train_test_split(df["TARGET"], train_size=0.80, shuffle=False, random_state=1,)

df_for_balancing = pd.concat([x_train, y_train], axis=1)
df_balanced = balance_df_by_target(df_for_balancing, "TARGET", method='over')

x_train_balanced = df_balanced.drop(["TARGET"], axis=1)
y_train_balanced = df_balanced["TARGET"]

In [51]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
      
        estimator.fit(x_train, y_train,
                      eval_set=[(x_train, y_train), (x_valid, y_valid)],
                      verbose=False
                      )
        y_train_pred = estimator.predict_proba(x_train)[:,1]
        y_valid_pred = estimator.predict_proba(x_valid)[:,1]
        # print(y_train_pred.shape, y_train.shape)
        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [52]:
cv_strategy = KFold(n_splits=5, random_state=1, shuffle=True)

lbgm_estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, lgbm_model, metric=roc_auc_score, cv_strategy=cv_strategy )

Fold: 1, train-observations = 70459, valid-observations = 17615
train-score = 0.776, valid-score = 0.7317
Fold: 2, train-observations = 70459, valid-observations = 17615
train-score = 0.8035, valid-score = 0.7299
Fold: 3, train-observations = 70459, valid-observations = 17615
train-score = 0.7751, valid-score = 0.7272
Fold: 4, train-observations = 70459, valid-observations = 17615
train-score = 0.7802, valid-score = 0.7156
Fold: 5, train-observations = 70460, valid-observations = 17614
train-score = 0.7696, valid-score = 0.7208
CV-results train: 0.7809 +/- 0.012
CV-results valid: 0.725 +/- 0.006
OOF-score = 0.7247


In [53]:
## Прогноз
y_pred = np.zeros(x_test.shape[0])

for estimator in lbgm_estimators:
    y_pred += estimator.predict_proba(x_test)[:,1]

y_pred = y_pred/cv_strategy.n_splits

score = roc_auc_score(y_test, y_pred)
print(f'Test predict = {score:.5}')

Test predict = 0.73636


In [54]:
cv_strategy = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

lgbm_estimators_stratify, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, lgbm_model, metric=roc_auc_score, cv_strategy=cv_strategy
)

Fold: 1, train-observations = 70459, valid-observations = 17615
train-score = 0.7813, valid-score = 0.7276
Fold: 2, train-observations = 70459, valid-observations = 17615
train-score = 0.7736, valid-score = 0.7213
Fold: 3, train-observations = 70459, valid-observations = 17615
train-score = 0.7665, valid-score = 0.7331
Fold: 4, train-observations = 70459, valid-observations = 17615
train-score = 0.7974, valid-score = 0.724
Fold: 5, train-observations = 70460, valid-observations = 17614
train-score = 0.7784, valid-score = 0.7199
CV-results train: 0.7794 +/- 0.01
CV-results valid: 0.7252 +/- 0.005
OOF-score = 0.7249


In [55]:
## Прогноз
y_pred = np.zeros(x_test.shape[0])

for estimator in lgbm_estimators_stratify:
    y_pred += estimator.predict_proba(x_test)[:,1]

y_pred = y_pred/cv_strategy.n_splits

score = roc_auc_score(y_test, y_pred)
print(f'Test predict = {score:.5}')

Test predict = 0.73457


In [56]:
cv_strategy = KFold(n_splits=5, random_state=1, shuffle=True)

catb_estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, catb_model, metric=roc_auc_score, cv_strategy=cv_strategy )

Fold: 1, train-observations = 70459, valid-observations = 17615
train-score = 0.8067, valid-score = 0.736
Fold: 2, train-observations = 70459, valid-observations = 17615
train-score = 0.7735, valid-score = 0.7267
Fold: 3, train-observations = 70459, valid-observations = 17615
train-score = 0.7619, valid-score = 0.7276
Fold: 4, train-observations = 70459, valid-observations = 17615
train-score = 0.783, valid-score = 0.7147
Fold: 5, train-observations = 70460, valid-observations = 17614
train-score = 0.7735, valid-score = 0.7194
CV-results train: 0.7797 +/- 0.015
CV-results valid: 0.7249 +/- 0.007
OOF-score = 0.7246


In [57]:
## Прогноз
y_pred = np.zeros(x_test.shape[0])

for estimator in catb_estimators:
    y_pred += estimator.predict_proba(x_test)[:,1]

y_pred = y_pred/cv_strategy.n_splits

score = roc_auc_score(y_test, y_pred)
print(f'Test predict = {score:.5}')

Test predict = 0.73555


In [58]:
# cv_strategy = KFold(n_splits=5, random_state=1, shuffle=True)

# catb_estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
#    x_train, y_train, catb_model, metric=roc_auc_score, cv_strategy=cv_strategy )

In [59]:
## Прогноз
y_pred = np.zeros(x_test.shape[0])

for estimator in catb_estimators:
    y_pred += estimator.predict_proba(x_test)[:,1]

y_pred = y_pred/cv_strategy.n_splits

score = roc_auc_score(y_test, y_pred)
print(f'Test predict = {score:.5}')

Test predict = 0.73555


In [60]:
# cv_strategy = KFold(n_splits=5, random_state=1, shuffle=True)

# lbgm_estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
#    x_train_balanced, y_train_balanced, lgbm_model, metric=roc_auc_score, cv_strategy=cv_strategy )

In [61]:
## Прогноз
y_pred = np.zeros(x_test.shape[0])

for estimator in catb_estimators:
    y_pred += estimator.predict_proba(x_test)[:,1]

y_pred = y_pred/cv_strategy.n_splits

score = roc_auc_score(y_test, y_pred)
print(f'Test predict = {score:.5}')

Test predict = 0.73555


In [62]:
# Обработать целевой датасет. вычислить результат. Загрузить на платформу.

In [63]:
test_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/test.csv')
test_df.head(4)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE
0,123724268,Cash
1,123456549,Cash
2,123428178,Credit Card
3,123619984,Cash


In [64]:
test = df_prepare(test_df)

final_id = test['APPLICATION_NUMBER']
x_data = test.drop(["APPLICATION_NUMBER"], axis=1)

In [66]:
# прогноз
y_pred = np.zeros(x_data.shape[0])

for estimator in catb_estimators:
    y_pred += estimator.predict_proba(x_data)[:, 1]
    
y_pred=y_pred/cv_strategy.n_splits

In [ ]:
y_true = np.zeros(x_data.shape[0])

In [68]:
df_final = pd.DataFrame({'APPLICATION_NUMBER':final_id, 'TARGET':list(y_pred)})

In [69]:
df_final.head(5)

,APPLICATION_NUMBER,TARGET
0,123724268,0.054297
1,123456549,0.261454
2,123428178,0.148470
3,123619984,0.085980
4,123671104,0.020987


In [ ]:
df_final.to_csv('prediction_best.csv', index=False, encoding='utf-8')

In [ ]:
df_final = pd.read_csv('prediction_best.csv')

In [ ]:
df_final.shape

(165141, 2)